In [1]:
import pandas as pd
from fastai.imports import *
from fastai.structured import *
#!pip install --user xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
import pandas as pd
import numpy as np


In [2]:
# Reading the bulldozers train.csv file
df_raw = pd.read_csv('/home/jupyter-mouguntu/Train.csv',low_memory=False, parse_dates=["saledate"])

In [4]:
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,Low,2008-12-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,High,2004-08-26,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,High,2005-11-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,Low,2009-08-27,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
9,1139275,65000,1016474,3883,121,3.0,1000,20700.0,Medium,2007-08-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [3]:
fld = df_raw.saledate

In [4]:
fld.dt.year

0         2006
1         2004
2         2004
3         2011
4         2009
5         2008
6         2004
7         2005
8         2009
9         2007
10        2008
11        2006
12        2005
13        2006
14        2006
15        2006
16        2007
17        2010
18        2006
19        2005
20        2006
21        2006
22        2007
23        2006
24        2004
25        2006
26        2006
27        2007
28        2007
29        2006
          ... 
401095    2011
401096    2011
401097    2011
401098    2011
401099    2011
401100    2011
401101    2011
401102    2011
401103    2011
401104    2011
401105    2011
401106    2011
401107    2011
401108    2011
401109    2011
401110    2011
401111    2011
401112    2011
401113    2011
401114    2011
401115    2011
401116    2011
401117    2011
401118    2011
401119    2011
401120    2011
401121    2011
401122    2011
401123    2011
401124    2011
Name: saledate, Length: 401125, dtype: int64

In [5]:
#expanding date-time into additional fields
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [6]:
#converting strings to pandas categories.
train_cats(df_raw)

In [7]:
df_raw.UsageBand

0            Low
1            Low
2           High
3           High
4         Medium
5            Low
6           High
7           High
8            Low
9         Medium
10        Medium
11           Low
12           NaN
13        Medium
14           Low
15        Medium
16           Low
17           NaN
18           Low
19        Medium
20        Medium
21        Medium
22           Low
23          High
24        Medium
25        Medium
26           Low
27           NaN
28           Low
29        Medium
           ...  
401095       NaN
401096       NaN
401097       NaN
401098       NaN
401099       NaN
401100       NaN
401101       NaN
401102       NaN
401103       NaN
401104       NaN
401105       NaN
401106       NaN
401107       NaN
401108       NaN
401109       NaN
401110       NaN
401111       NaN
401112       NaN
401113       NaN
401114       NaN
401115       NaN
401116       NaN
401117       NaN
401118       NaN
401119       NaN
401120       NaN
401121       NaN
401122       N

In [8]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [9]:
#specify the order to use for categorical variables 
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [10]:
df_raw.UsageBand = df_raw.UsageBand.cat.codes

In [11]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/xgb_bulldozer-raw')
#df_raw = pd.read_feather('tmp/xgb_bulldozer-raw')

In [2]:
df_raw = pd.read_feather('tmp/xgb_bulldozer-raw')

/home/jupyter-mouguntu/.local/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [3]:
#proc_df for converting categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.
df, y,nas = proc_df(df_raw, 'SalePrice')

In [ ]:
data_dmatrix = xgb.DMatrix(data=df,label=y)

In [ ]:
# splitting data into train-test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=123)

In [7]:
## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [4]:
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

In [9]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
xg_reg = xgb.XGBRegressor()

In [11]:
random_search=RandomizedSearchCV(xg_reg, params, n_jobs=1,cv = 25 )

In [15]:
from datetime import datetime
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable

[13:11:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:12:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:13:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:14:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:16:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:17:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [16]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.4, gamma=0.0,
       importance_type='gain', learning_rate=0.15, max_delta_step=0,
       max_depth=12, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [8]:
#Fitting best estimator to xgb
xg_reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.4, gamma=0.0,
       importance_type='gain', learning_rate=0.15, max_delta_step=0,
       max_depth=12, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [9]:
# Fitting the data to XGB
m = xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

[14:33:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [11]:
#predicted y_test values
preds

array([13048.472, 16185.535, 11629.463, ..., 28921.38 , 22983.361, 36187.387], dtype=float32)

In [12]:
# Root mean squared log error
display(y_test, preds)
rmse = np.sqrt(mean_squared_log_error( y_test, preds ))
print("RMSE: %f" % (rmse))

array([15000, 13000, 12500, ..., 27500, 21000, 40000])

array([13048.472, 16185.535, 11629.463, ..., 28921.38 , 22983.361, 36187.387], dtype=float32)

RMSE: 0.209866


In [15]:
def rmsle(predicted, real):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

def print_score(m):
    res = [rmsle(m.predict(X_train), y_train), rmsle(m.predict(X_test), y_test),
                m.score(X_train, y_train), m.score(X_test, y_test)]
    print(res)
print_score(m)

[0.1536923478956337, 0.2098664294101122, 0.9658753949460054, 0.9155265351024242]
